In [1]:
from IPython import get_ipython
ipython = get_ipython()
ipython.run_line_magic("load_ext", "autoreload")
ipython.run_line_magic("autoreload", "2")

In [2]:
import einops
from functools import partial
import torch
import datasets
from torch import Tensor
from torch.utils.data import DataLoader
from datasets import load_dataset, concatenate_datasets
from jaxtyping import Float, Int, Bool
from typing import Dict, Iterable, List, Tuple, Union
from transformer_lens import HookedTransformer
from transformer_lens.utils import get_dataset, tokenize_and_concatenate, get_act_name, test_prompt
from transformer_lens.hook_points import HookPoint
from tqdm.notebook import tqdm
import pandas as pd
from circuitsvis.activations import text_neuron_activations
from utils.store import load_array, save_html, save_array, is_file, get_model_name, clean_label, save_text
from utils.circuit_analysis import get_logit_diff

from utils.tokenwise_ablation import (
    compute_ablation_modified_logit_diff,
    load_directions,
    get_random_directions,
    get_zeroed_dir_vector,
    get_layerwise_token_mean_activations
)
from utils.datasets import OWTData

## Comma Ablation on Natural Text

### Model Setup

In [3]:
device = torch.device("cuda")
MODEL_NAME = "EleutherAI/pythia-2.8b"
TOKEN_ID = 13
BATCH_SIZE = 16

In [4]:
torch.set_grad_enabled(False)
model = HookedTransformer.from_pretrained(
    MODEL_NAME,
    center_unembed=True,
    center_writing_weights=True,
    fold_ln=True,
    refactor_factored_attn_matrices=False,
    device=device,
)

Using pad_token, but it is not set yet.


Loaded pretrained model EleutherAI/pythia-2.8b into HookedTransformer


### Data

In [5]:
owt_data = OWTData.from_model(model)
owt_data.preprocess_datasets(token_to_ablate=TOKEN_ID)

Repo card metadata block was not found. Setting CardData to empty.


In [6]:
datasets = owt_data.dataset_dict

In [7]:
datasets['train']

Dataset({
    features: ['tokens', 'attention_mask', 'positions', 'has_token'],
    num_rows: 10959
})

In [8]:
datasets['train'][0]['tokens'][30:40]

tensor([  249, 46882,    71,   668,   275,  6176,    13,   533,   253,  2208])

In [9]:
print(model.to_str_tokens(datasets['train'][0]['tokens'][30:40]))

['in', ' Kamp', 'f', '”', ' in', ' Germany', ',', ' but', ' the', ' government']


In [10]:
print(datasets['train'][0]['positions'][30:40])

tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 0])


In [11]:
dataloader = owt_data.get_dataloaders(batch_size=BATCH_SIZE)['train']

In [12]:
len(datasets['train']), len(dataloader)

(10959, 685)

### Calculate Means

In [13]:
comma_mean_values = get_layerwise_token_mean_activations(model, dataloader, token_id=13, device=device)
save_array(comma_mean_values, 'comma_mean_values.npy', model)

  0%|          | 0/685 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# load the files
owt_mean_values = torch.from_numpy(load_array('comma_mean_values.npy', model)).to(device)

### Get Loss Results

In [ ]:
heads_to_ablate = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]
heads_to_freeze = [(layer, head) for layer in range(model.cfg.n_layers) for head in range(model.cfg.n_heads)]
layers_to_ablate = [layer for layer in range(model.cfg.n_layers)]
ablated_loss_diff, orig_loss = compute_mean_ablation_modified_loss(
    model, 
    balanced_data_loader,
    layers_to_ablate,
    comma_mean_bal_values,
    debug=False
)

ablated_loss = orig_loss + ablated_loss_diff

# orig_accuracy = (orig_ld_list > 0).float().mean()
# ablated_accuracy = (ablated_ld_list > 0).float().mean()
# freeze_ablated_accuracy = (freeze_ablated_ld_list > 0).float().mean()

# print(f"Original mean logit diff: {orig_ld_list.mean():.4f}")
# print(f"Original accuracy: {orig_accuracy:.4f}")
# print("\n")
# print(f"Comma-ablated mean logit diff: {ablated_ld_list.mean():.4f}")
# print(f"Comma-ablated accuracy: {ablated_accuracy:.4f}")
# print(f"Percent drop in logit diff with comma ablation: {(orig_ld_list.mean() - ablated_ld_list.mean()) / orig_ld_list.mean() * 100:.2f}%")
# print(f"Percent drop in accuracy with comma ablation: {(orig_accuracy - ablated_accuracy) / orig_accuracy * 100:.2f}%")
# print("\n")
# print(f"Attn frozen, comma-ablated mean logit diff: {freeze_ablated_ld_list.mean():.4f}")
# print(f"Attn frozen, comma-ablated accuracy: {freeze_ablated_accuracy:.4f}")
# print(f"Percent drop in logit diff with attn frozen, comma ablation: {(orig_ld_list.mean() - freeze_ablated_ld_list.mean()) / orig_ld_list.mean() * 100:.2f}%")
# print(f"Percent drop in accuracy with attn frozen, comma ablation: {(orig_accuracy - freeze_ablated_accuracy) / orig_accuracy * 100:.2f}%")
# print("---------------------------------------------------------")
# print("Random direction ablation results:")
# print(f"Comma-ablated mean logit diff: {ablated_ld_list_rand.mean():.4f}")
# print(f"Percent drop in logit diff with comma ablation: {(orig_ld_list_rand.mean() - ablated_ld_list_rand.mean()) / orig_ld_list_rand.mean() * 100:.2f}%")
# print(f"Attn frozen, comma-ablated mean logit diff: {freeze_ablated_ld_list_rand.mean():.4f}")
# print(f"Percent drop in logit diff with attn frozen, comma ablation: {(orig_ld_list_rand.mean() - freeze_ablated_ld_list_rand.mean()) / orig_ld_list_rand.mean() * 100:.2f}%")

In [ ]:
ablated_loss_diff[0]

In [ ]:
ablated_loss_diff[0][33], orig_loss[0][33], ablated_loss[0][33]

In [ ]:
batch = next(iter(balanced_data_loader))

In [ ]:
model.to_str_tokens(batch['tokens'][0][33])

In [ ]:
batch

In [ ]:
model.reset_hooks()